# Capstone Project
## Machine Learning Engineer Nanodegree
Robert McKee  
September 30, 2016

## I. Definition


### Project Overview

I wanted to ask if I could properly classify patents based on the text from their description.
   - Classify: each patent is labeled with a classification number. One for the US, one for Patent Cooperation Treaty, and a new class that is designed to merge the two. There are over 400 main classifications and thousands of sub classes. The USPTO pays a company to classify each incoming patent, and re-classify it if it is incorrectly classified. Humans perform this activity and use the claims and description to determine the class the patent belongs to.
    - Description: This is the long text portion of the patent that explains how to make the technology and supporting evidence that the technology works and the inventor created it. That is the best way for me to describe it to someone who doesn't know about patents.

Currently the US Patent office (USPTO) pays a contractor to classify patents. This process started in 2006 and it is estimated that 10% of patents are incorrectly classified by the contractor, while they are charging the government around $20 million per year for the service. I gathered data from 2005 when the USPTO was still classifying patents. It was not in a format ready for machine learning algorithms.

### Problem Statement

The problem as stated in the industry is that the patent office contractor misclassifies around 10% of patent application. This leads to months of delays and increased costs for getting a patent granted. This performance level is also very costly. Over 5 years the USPTO will pay $95 million for an outside contractor to perform at this level.

Strategy: use the bulk data downloads available from the USPTO to gather patent data from 2005. Then parse the description (input) and class (target) data. Once the data set is ready, I will then apply a plurality of supervised learning algorithms for training the machine learning model.


### Metrics

Metrics for analyzing the algorithms were an afterthought. I was having so much trouble collecting and cleaning up the data for processing that I didn't even think about analyzing the data much less analyzing the algorithm performance. Once I had clean data and was easily able to process the data in algorithms, I found and used metrics.classification_report() to see the precision, recall, and f1-score for each algorithm. 

Precision (also called positive predictive value) is the fraction of retrieved instances that are relevant, while recall (also known as sensitivity) is the fraction of relevant instances that are retrieved (see diagram below). [(source)](https://en.wikipedia.org/wiki/Precision_and_recall).
![Precision Formula](precisionformula.svg)
![Recall Formula](recallformula.svg)

The traditional F-measure or balanced F-score (F1 score) is the harmonic mean of precision and recall:![F-scoure](https://wikimedia.org/api/rest_v1/media/math/render/svg/2a535af958c5c05eb6c41ebac60a81dbddb29e61) 

![Precision and Recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/400px-Precisionrecall.svg.png)


## II. Analysis



My intention was to train a classifier to correctly classify the top 10 classes that area still relevant today. I utilized statistics on the classifications with the most patents each year [source](http://www.uspto.gov/web/offices/ac/ido/oeip/taf/data/cbcby.htm). I limited it to the top 10 classes in the post processing stage once I realized how large the data files would be. Also, because the instructions for the project mentioned limiting the size of files so the grader wouldn't have to wait for large files to download.


Based on that info I will narrow my focus to the top 10 classes of 2015, which appear in the top 40 of 2005:


Class Name | Class Number (US)
------------ | -------------
Active Solid-State Devices (e.g., Transistors, Solid-State Diodes) 	| 257
Multiplex Communications	| 370
Telecommunications	| 455
Computer Graphics Processing and Selective Visual Display Systems	| 345
Drug, Bio-Affecting and Body Treating Compositions	| 514
Electrical Computers and Digital Processing Systems: Multicomputer Data Transferring	| 709
Television	| 348
Drug, Bio-Affecting and Body Treating Compositions	| 424
Data Processing: Database and File Management or Data Structures	| 707
Data Processing: Vehicles, Navigation, and Relative Location	| 701



Sorted by most occuring in 2015
![Image of excel spreadshit sorting 2015 classes](classes-sorted-2015.png)



Sorted by most occuring in 2005
![Image of excel spreadshit sorting 2005 classes](classes-sorted-2005.png)



The size of the data set:
>RangeIndex: 19439 entries, 0 to 19438

>Data columns (total 5 columns):

>claims         19439 non-null object

>usMainClass    19439 non-null category

>description    19439 non-null object

>artUnit        19439 non-null category

>patnum         19439 non-null object

>dtypes: category(2), object(3)

>memory usage: 515.3+ KB



For the text in the description I lowercased each word, used regex to select for only letters, thus removed any numbers, removed stop words with NLTK, and rejoined the data for the description feature as a long continuous string. I did not stem the data because I felt that it would reduce the data to a large degree. I say that because in patent law, the inventor can be his own lexicographer. Meaning, the inventor can define words in the description as he sees fit. So I wanted to leave some wiggle room with stem words for inventors to use them in different ways. (code below)


>   from nltk.corpus import stopwords

>   stops = set(stopwords.words('english'))

> 	description = re.findall ( '<description id="description">(.*?)</description>', patent, re.DOTALL)

> 	description = description[0]

> 	description = description.lower()

>	letters_only = re.sub("[^a-zA-Z]", " ", description)

>	words = letters_only.split()

>	meaningful_words = [w for w in words if not w in stops]

>	new_description = " ".join( meaningful_words )

>	patdict['description'] = new_description


To check for missing data I used the isnull() function in pandas. I iterated through each row of the description and usMainClass columns. If a row returned True based on isnull(), then it would add 1 to the count. At the end of the script it would print the total count. No missing data was detected.
![null data](null_data.png)


### Exploratory Visualization
My go to data visualization when first working with data is the .head() function. Seeing the data in tabular format lets me get an idea if anything is in the wrong format, if any data is improperly parsed, or even not in the data set at all.
![to visualize the data formats](data_head.png)

Next, it was important for me to verify if only the classes I wanted would be in the data set so that I am sure I will be training the algorithm on the classifications I want to predict. To see this, I used the .unique() function to see each unique class inside the usMainClassification or target column.
![to see what classes are in the data set](usMainClass_unique.png)

![Histogram of Classes](exploreHist.png)

### Algorithms and Techniques

While listening to the lectures, one statement stood out to me during the housing price prediction project. The instructor said this type of problem in the real world is easily done by a professional realator with years of experience who can quickly give you a range for a price based on his knowledge. But for a new realtor, they would need to see many houses sold before they could recognize what price belongs to what house. That reminded me of patent classification. In one article I remember an attorney saying sometimes you just know what class the patent belongs to based on experience. Otherwise, finding the right class for a patent can be challenging. So I looked at this as a classification problem using supervised learning.

This is a supervised learning issue because I know what classes or labels the data needs to fit into, and I have historical data to train the algorithm with. My only problem was how to give the algorithm the right data, in the right format for it to be trained properly.

Naive Bayse performs well on categorical data. While it does not do well if a category is in the test data and not in the training data, I don't expect any "new" categorical data to appear in the test data. I am trimming down the data set for training and testing to only include a small set of classes. This is similar to the real world in that there are examples for every classification, otherwise the classification would not exist, because they create a classification for patent applications as needed.[source](https://www.analyticsvidhya.com/blog/2015/09/naive-bayes-explained/).

Linear SVM is often recommended for text classification because text is often linearly separable, and it can handle many features, which is a characteristic of text data sets.[source](http://www.svm-tutorial.com/2014/10/svm-linear-kernel-good-text-classification/)

KNN: KNN can be great for predicting certain datasets, however, it's strengths are with numerical data. The data I am working with is not currently in a numerical format. I know that will be an issue, and I could come up with a system to represent the text as numerical data, but I want to experiment mainly using categorical and string datatypes.

cons:

### Benchmark

My real world benchmark is an estimation from a long time patent attorney who said that from his analysis he estimates that atleast 10% of the classifications by the contracting company are wrong.[citation](http://www.ipwatchdog.com/2014/03/11/when-uspto-classifies-an-application-incorrectly/id=48457/). I would say that the current process 90% precision in that of the classifications made they get 90% correct but 10% are false positives for a class.  They are peing paid close to $20 million per year for this. 

My benchmark is attempting to perform at or above 90% F-score in classifying patents. Equalling a 90% F-score for a program that has taken a few weeks and essentially free compared to the current option would be a massive improvement for the government. I am using F-score because I need to balance both recall and precision. Recall becase I need to correctly classify as many in the data set as I can, but I have to balance that with precision by not having too many false positives. I am focusing on F-score since ultimately I want to classify every patent into every classification. Currently I am limiting my data set and not classifying all of the patents, meaning I can have false negatives for a class. When I attempt to classify every patent into all available classifications, then I not have false negatives overall since each patent will be classified, so it will theoretically be a false positive. When analyzing at the class level, I could have false negatives. But I am looking at correctly classifying every patent, so precision and recal matter, thus, F-Score is my benchmark.




## III. Methodology


### Data Preprocessing

Most of my time was spent collecting and processing data. I downloaded the xml data from the USPTO bulk download page. To download bulk patent data I used the [USPTO bulk download site](https://pairbulkdata.uspto.gov/). 2005 had 52 packets averaging around 45 mb each. 2015 also had 52 packets but averaged around 150mb each. Once I realized the size of the data, I limited my application and analysis to 2005 only. This decision was to make it easier for theh grader to download any necessary files.

Once I downloaded the files I then tried to use xml parsing to clean up the data. However, after numerous issues ultimately leading to problems implementing the machine learning algorithms, I discovered the xml file was what some called "poisoned" because in using xml parsing I was only able to parse one patent per file. I should have been parsing around 3,000 patents per file. 

I am not going to imply that I discovered this in one process of gathering, processing, and analyzing the data. I went through multiple iterations of the entire process. 3 to be exact. Each time I started over, the process was faster, the code was more efficient, and the results were better. 

The first iteration of processing the data utilizing xml parsing resulted in success on a small test sample I used to run my code before running the code on the larger data files. However, when I ran the code on the large files, I only got 1 patent per file. Additionally, in the first iteration, my tokenization process was involved in the parsing process and that resulted in the program taking 8+ hours to run on 1.6gb of text data. The first time I ran that process, I left out a line of code that wrote the dictionary results (each patent was parsed into a dictionary) into a list for use as a csv file. So my output after 8 hours was nothing. That was a hard learning experience. But even after getting a result saved as a csv file, the resulting data was in a format that kept resulting in errors during machine learning. So I made the decision to start over. From scratch.

In the next process I first filtered the patents for only patents that were in the top 10 classes I was searching for. Next, I limited what data I parsed from the patents, and I left out the word_tokenize function of NLTK. This allowed the program to return a result in less than 30 minutes.

This process allowed me to run machine learning algorithms but I was not getting results that I thought were impressive. So I went back and parsed the data from scracth. It turned out my regex code was missing the specific location for main classification of the patent and finding other positions that were labeled main classification but using a different classification style. This change resulted in the data going from approximately 13,000 patents that were not necessarily in the classifications I was looking for, to 19,000 patents that were in the classifications I was looking for.  

Overall, processing the data and limiting my search scope brought the data size down from 11gb to 900mb.


### Implementation

I utilized numerous supervised learning methods for classification. I did not go in expecting one method to be superior to the others, I wanted to try numerous methods and find the best performing algorithm based on the data. I used Naive Baise, Linear SVM, and K-nearest neighbors classifier. To anylize the performance of the algorithms I used the metrics.classification_report(). 

Applying the algorithms was realatively easy and fast. I ran then numerous times and all while on a plane without internet connection to look for answers to problems. The majority of my time on this project was spent gathering the data, and putting it into a clean format for the machine learning algorithms to easily utilize. Once the data was clean, I could quickly try out multiple algorithms.

### Refinement

Most of my process was a manual refinement of the data input, and what data I used. This was based on my knowledge of the data, the method used by the people examining these applications to classify them, and my understanding of how people write the patents in order get the patent passed by the USPTO. I applied a lot of cynicism that ultimately seemed to return a positive result in the F-Score. One method I used to refine the results was to convert the usMainClass or the target classification/labels from a datatype int64 to categorical. This, from my understanding, then allowed the machine learning algorithms to recognize them as distinct labels. Before the data was just numerical and seen from a numbers perspective, with the processing favoring statistical/numerical results instead of a more classification result oriented process. 

I started out using the suggested variables for the Linear SVM from the [instructions on working with text data from the scikit documentation](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html). Going back, I got rid of the variables to get a baseline of performance of the standard Linear SVM. Removing the variables actually improved the results for class 257 in the np.mean, the precision, and F1-Score, the recall did go down slightly. However, as stated earlier, for the overall project of classifying every patent, I am going for a higher F1-Score, so this is a good result.

metric | Variables from sklearn documentation | Removal of variables
-----------------------------------------------------------
np.mean |0.7009 | 0.719
precision |0.90| 0.94
recall |0.99 | 0.97
F1-Score | 0.94 | 0.96



## IV. Results

### Model Evaluation and Validation

The results of the Linear SVM were the best and met my benchmark standard but only for class 257. Class 257 had 1,380 data points. This was double the next most common class. From this and from the fact that this result was obtained from the data set containing 6,000 more patents, it appears that a larger data set is required for the other classes to increase performance.  

>Class 257 with Linear SVM returned a precision score of 0.90, recal of 0.99, and f1-score of 0.94. This >consistency above 90 for all factors was not seen in any other class. 

### Validation discussion
**Question from Reviewer:** Also the precision, recall and F1 scores reported, were these scores as a result of testing on a separate test data? or on the training data?
The precision, recal, and F1 scores reported were a result of testing on a separate test data created through a train test split as described below ....


**Question from Reviewer:** How was the data split? is the method selected appropriate given the target class? meaning, is there class imbalance? does the validation method for example, stratify to account for class imbalance (if any)? 
Me: That is a good question. I didn't think about that. Going back to look at them, the train test split  are similar in makeup based on the below histogram and discussed further in the Train Test Split Discussion.

**Total data distribution**
![Total data distribution](totalclassdistribution.png)

**Training set distribution**
![Training set distribution](trainclassdistribution.png)

**testing set distribution**
![testing set distribution](testclassdistribution.png)

### Train Test Split Discussion
The full dataset was named "data". While that data set is informative, to create the predictive model and rule out overfitting I needed to split the data into a training and testing set. Overfitting can occur a few ways, but the splitting of data into a training and testing set is designed to make sure the algorithm doesn't see the data you will later tests its prediction abilities on. To not separate the data in the train test split is similar to giving a student the test with answers to study with, then testing them using the test they have already seen, and using their performance as an indicator of how well they know and understand the material. In reality, the student only knows answers and can't apply that knowledge to other situations. In the machine learning algorithm, the algorithm would be able to correctly classify each data point we give it to predict that was in the original data, however, if we gave it data that it had never seen before, the results would likely be poor because it will have trouble applying it's knowledge of answers to a data point where it doesnt' know the answer.

I split up the data in a few steps, primarily for my own understanding of how the data was being processed, and not for the sake of efficiency or any programming rules. I first split the data into a training set and a data set: 
> train, test = train_test_split(data, test_size = 0.3)
Then I looked at the data to get an understanding of what was in there and to look for any glaring problems.

>train['usMainClass'].value_counts()
>train['usMainClass'].value_counts().plot(kind='bar')
>test['usMainClass'].value_counts()
>test['usMainClass'].value_counts().plot(kind='bar')

Looking at the .value_counts() I could see that the make up of categorical data was uniformly distributed into both the training set and the testing set, with class 257 representing the largest group and 701 being the smallest.

Next, I split up each training and testing data set into an input and a target. The input is the data I gave to the algorithm to use to predict from. It is like the question on the test. The target is the data or answer I expect from the algorithm. I deviated from typical nomenclature by using the term input but it made a lot more sense to me about the data it represented. 

The easier and more efficient method for accomplishing the train test split is to use this line of code:
> X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=0)
That is from the sklearn documentaion and does not represent data names for this project.

------------------------------------------------------------------------------------------

#### Naive Bayse Results
![Naive Bayse Results](NBResults.png)

#### SVM Results
![SVM Results](SVMResults.png)

#### K-nearest neighbors Results
![K-nearest neighbors Results](KNNResults.png)

### Justification

While the performance result for class 257 is better than the benchmark 90% correctly classified by humans, estimated by a patent lawyers, I was hoping to get that result for all of the classes. However, what the data tells me is that it is possible to obtain the performance I am expecting, I just need to download more data. The download times of the server holding the USPTO data are the limiting factor. While downloading at a fiber optic connection with 90mb down/up, I could only download patent data at 1-2mb/s. 


## V. Conclusion

### Free-Form Visualization
Using the metrics.classification_report() I was able to visualize how important a large data set was to the accuracy of the algorithms. In the data below, class 257 performs well in precision, recall, and f1-score. What is clearly visible is that it has more support/data points that the other classes. This might not have been so obvious had its support number not been 4 digits while the other classes were only 3 digits. Still, visually it jumps out.

print(metrics.classification_report(testTarget, svm_predicted))
metrics.confusion_matrix(testTarget, svm_predicted)
             precision    recall  f1-score   support

        257       0.90      0.99      0.94      1380
        345       0.68      0.66      0.67       673
        348       0.67      0.53      0.59       380
        370       0.59      0.73      0.66       699
        424       0.69      0.27      0.38       362
        455       0.60      0.58      0.59       560
        514       0.69      0.94      0.80       658
        701       0.68      0.75      0.71       245
        707       0.52      0.41      0.46       278
        709       0.58      0.44      0.50       597

avg / total       0.70      0.70      0.69      5832

![Visualization of Metrics](classificationreportviz.png)

### Reflection

This project has mainly resulted in more questions that I have. Such as, how do I process larger data sets, how can I utilize off-site servers for processing large amounts of data instead of being limited by my laptops processor. While I was expecting to easily classify every patent class with ease, the difficulty in coming to a result of classifying one class has humbled me and given me a drive to learn more about scaling up this process.

I do think this process could be used in a general application. If not for the uspto, I can scale this up with more data and use invention disclosures from inventors to determine what class it would belong to and then use the predicted classification to conduct a search for similar patents. This would increase my efficiency at work.

### Improvement

To enable my implementation to be applied more generally to all patent classifications I will need alot more data points for each patent class. This will take a while to download the data from the USPTO. However, a friend has sent me 65gb of patent data covering 10+ years in a mongo database. Utilizing python libraries I can convert that database into a csv file for easy use in machine language programs to predict each patent classification. 

I definitely think a better solution exists. I think the increase in F-Score resulting from the increase in patents used as a data set is proof that I can increase F-Score. But from an algorithm point, I would like to put more time into learning tensorflow and utilize n-grams and other text analysis to perform more advanced text feature extraction. I would be interested to see how the tensorflow neural network applications affect the F-Score.
